<h1>Experience vélo à Saint Denis</h1>

In [213]:
import pandas as pd
import folium
# pip install geopandas
import geopandas as gpd
from shapely.geometry import Point
from IPython.display import display
import plotly.graph_objects as go
from bs4 import BeautifulSoup

1. La balade

In [214]:
photo= pd.read_csv('Data/Photo.csv')
# Directory where images are stored
image_directory = "Photo Saint-Denis/"

# Convert the pandas DataFrame to a GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(photo['Longitude'], photo['Latitude'])]
gdf = gpd.GeoDataFrame(photo, geometry=geometry, crs="EPSG:4326")

# Create a base map centered on the mean of the coordinates
m = folium.Map(location=[gdf['Latitude'].mean(), gdf['Longitude'].mean()], zoom_start=14, tiles='Stadia.OSMBright')

# Add points to the map
for idx, row in gdf.iterrows():
    # Construct the full path to the image
    image_path = f"{image_directory}{row['Filename']}"

    # HTML for the popup with the image
    popup_content = f"<img src='{image_path}' width='200'>"

    # Add a marker with the popup
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=250)
    ).add_to(m)

# Save the map to an HTML file
m.save('map.html')

# Display the map (if in a Jupyter Notebook)
display(m)



In [215]:


fig = go.Figure([go.Scatter(x=photo['Time Taken'], y=photo['Altitude'])])
fig.show()

La plus part des photo a été pris au cour de cette balade se concentrer sur l'infrastructure du vélo. Il port sur l'aménagement du vélo. Dans cet exposition, je vais présenter l'état actuelle d'infrastrucutre du vélo sur la tétiroire de Saint-Denis et AUbervillier

2. Vélo route en Ile De France

In [216]:
route_velo = gpd.read_file('Data/reseau-vif-velo-ile-de-france.geojson')
route_velo = route_velo.to_crs("EPSG:2154")
route_velo['distance'] = route_velo.length

Skipping field j_num_ligne: unsupported OGR type: 5


In [218]:


def extract_tag(html):
    if not isinstance(html, str):
        return []
    soup = BeautifulSoup(html, 'html.parser')
    return [li.text.strip() for li in soup.find_all('li')]

route_velo['route_velo'] = route_velo['numero_ligne'].apply(extract_tag)
route_velo = route_velo.explode('route_velo')

In [219]:
route_velo.explore(column='route_velo',
                   cmap="tab10",
                   categorical=True,
                   legend=True,
                   tiles='CartoDB positron',
                   figsize=(8, 8))

In [220]:
# 1. Group by BOTH 'route' and 'status'
# This automatically creates a new line for every unique status a route has
velo = route_velo.groupby(['route_velo', 'niveau_valid_amenag'], as_index=False)['distance'].sum()

# 2. Rename columns to match your desired output
velo.columns = ['route', 'status', 'distance']

# 3. Sort Naturally (V2 before V10)
# We make a temporary helper column to turn "V10" into the number 10
velo['sort_helper'] = velo['route'].str.slice(1).astype(int)

# Sort by the helper number first, then by status
velo = velo.sort_values(by=['sort_helper', 'status'])

# Drop the helper column now that we are done
velo = velo.drop(columns='sort_helper')

velo

,route,status,distance
0,V1,En travaux,3714.037456
1,V1,Mis en service,41165.657527
2,V1,Pas d'information,5593.588461
3,V1,À l'étude,7044.005593
7,V2,En travaux,118.011912
8,V2,Mis en service,36657.436123
9,V2,Pas d'information,3697.416358
10,V2,À l'étude,10626.671813
15,V3,En travaux,3946.985699
16,V3,Mis en service,50036.582084


In [221]:
import plotly.express as px
fig = px.pie(velo , values='distance', names='status', title='VOie par type')

fig.show()

In [222]:
fig = px.bar(velo, x='route', y='distance', color='status', title='Status travaux')
fig.show()

In [223]:
point = [48.912196, 2.373194]
from shapely.geometry import Point

centroid = gpd.GeoSeries(
    Point(point[1], point[0]),
    crs="EPSG:4326"
)

centroid = centroid.to_crs("EPSG:2154")
buffer = centroid.buffer(5000)

route_velo_Saint_Denis = route_velo.to_crs("EPSG:2154")
route_velo_Saint_Denis = route_velo_Saint_Denis[route_velo_Saint_Denis.intersects(buffer.iloc[0])]

route_velo_Saint_Denis = route_velo_Saint_Denis.to_crs(epsg=4326)
buffer = buffer.to_crs(epsg=4326)



In [224]:
m = route_velo_Saint_Denis .explore(
    column="route_velo",
    categorical=True,
    cmap="tab10",
    legend=True ,
    location=point,
    tiles='Stadia.OSMBright',
    zoom_start=14
)



m

Section 3 : Type d'aménagement

In [225]:
piste_cyclable = gpd.read_file('Data/amenagements-velo-en-ile-de-france.geojson')
zone_interest = piste_cyclable[piste_cyclable['nom_com'].isin([ 'Aubervilliers', 'Saint-Denis'])]

In [226]:
mask = (
    (zone_interest['ad'] != zone_interest['ag']) &
    (zone_interest['ad'].notna()) &
    (zone_interest['ag'].notna())
)
zone_interest.loc[mask, 'longueur'] = zone_interest.loc[mask, 'longueur'] / 2
zone_interest[
    (zone_interest['ad'] != zone_interest['ag']) &
    (zone_interest['ad'].notna()) &
    (zone_interest['ag'].notna())]

/tmp/ipykernel_50549/3136350160.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 24.5  42.5  16.5  42.   13.5  67.5 103.   11.5  21.5  40.5  59.5   6.
  18.   68.   14.5  27.   58.5 132.  142.5   4.5  35.   66.5 124.   70.
   5.   92.5  28.   92.5   0.    7.   42.5]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.



,geo_shape,osm_id,highway,sens_voit,revetement,panneaux,ad,ag,nv,moyenn_ech,petite_ech,insee_com,nom_com,nom_voie,longueur,notes,geo_point_2d,geometry
5786,None,4.632588e+08,residential,UNIQUE,asphalt,None,cheminement uni,DSC bande,None,22,2,93066,Saint-Denis,Rue Camille Moke,24.5,None,"{ ""lon"": 2.3494689999999947, ""lat"": 48.9227438...","LINESTRING (2.34956 48.92253, 2.34938 48.92296)"
6317,None,1.129100e+07,residential,UNIQUE,asphalt,None,cheminement uni,DSC bande,limite 30,22,2,93066,Saint-Denis,Rue Paul Éluard,42.5,None,"{ ""lon"": 2.3454291027211287, ""lat"": 48.9393743...","LINESTRING (2.3449 48.93949, 2.34494 48.93944,..."
15175,None,6.366149e+08,secondary,DOUBLE,asphalt,None,piste uni,cheminement uni,None,12,1,93066,Saint-Denis,Rue du Landy,16.5,None,"{ ""lon"": 2.3384066569236328, ""lat"": 48.9149585...","LINESTRING (2.3386 48.91488, 2.33851 48.91492,..."
15244,None,1.129105e+07,residential,UNIQUE,sett,None,cheminement uni,DSC bande,limite 30,22,2,93066,Saint-Denis,Rue Gisquet,42.0,None,"{ ""lon"": 2.3464426036438195, ""lat"": 48.9385717...","LINESTRING (2.347 48.93849, 2.34598 48.93864, ..."
22897,None,1.063576e+08,residential,UNIQUE,asphalt,None,piste uni,DSC,None,12,1,93066,Saint-Denis,Rue Guynemer,13.5,None,"{ ""lon"": 2.3633202, ""lat"": 48.946008050000003 }","LINESTRING (2.36322 48.94611, 2.36342 48.9459)"
22993,None,1.011752e+09,residential,UNIQUE,asphalt,None,cheminement uni,DSC,None,34,3,93066,Saint-Denis,Avenue Henri Rol-Tanguy,67.5,None,"{ ""lon"": 2.3644129144280321, ""lat"": 48.9178174...","LINESTRING (2.3635 48.91783, 2.36359 48.91783,..."
35956,None,1.083109e+08,residential,DOUBLE,asphalt,None,bande uni,piste uni,None,12,1,93066,Saint-Denis,Avenue des Fruitiers,103.0,None,"{ ""lon"": 2.3548183018622724, ""lat"": 48.9162637...","LINESTRING (2.35471 48.91719, 2.35472 48.91714..."
35996,None,1.391808e+09,residential,UNIQUE,asphalt,None,cheminement uni,DSC bande,None,22,2,93066,Saint-Denis,Rue Camille Moke,11.5,None,"{ ""lon"": 2.34959015, ""lat"": 48.922430399999996 }","LINESTRING (2.34962 48.92233, 2.34956 48.92253)"
36002,None,6.733521e+08,primary,UNIQUE,asphalt,None,cheminement uni,bande uni,None,22,2,93066,Saint-Denis,Boulevard de la Libération,21.5,None,"{ ""lon"": 2.3418246315312925, ""lat"": 48.9264954...","LINESTRING (2.34188 48.92631, 2.34186 48.92638..."
36044,None,1.005019e+09,residential,UNIQUE,sett,None,cheminement uni,DSC bande,limite 30,22,2,93066,Saint-Denis,Rue Dézobry,40.5,None,"{ ""lon"": 2.348288686807253, ""lat"": 48.93858662...","LINESTRING (2.34819 48.93823, 2.34821 48.93832..."


In [227]:
def combine_cols (row, col1, col2) :
    value1,value2 = row[col1], row[col2]
    if pd.isnull(value1) & pd.isnull(value2) :
        return None
    elif pd.isnull(value1) :
        return value2
    elif pd.isnull(value2) :
        return value1
    elif value1 == value2 :
        return value1
    else :
        return [value1,value2]

zone_interest['type voie'] = zone_interest.apply(combine_cols,args=('ad','ag'), axis=1)
zone_interest = zone_interest.explode('type voie')


/home/lecaotunglam/anaconda3/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [228]:
replacement_map = {
    'bande bi': "voie partage avec d'autre véhicule sans protection et priorité",
    'bande uni': "voie partage avec d'autre véhicule sans protection et priorité",
    'DSC': "voie partage avec d'autre véhicule sans protection et priorité",
    'DSC bande': "voie partage avec d'autre véhicule sans protection et priorité",
    'DSC piste': "voie partage avec protection et priorité",
    'piste bi': "voie partage avec protection et priorité",
    'piste uni': "voie partage avec protection et priorité",
    'voie bus bi': "voie partage avec protection et priorité",
    'voie bus uni': "voie partage avec protection et priorité",
    'voie verte uni': "voie partage piéton",
    'piste trottoir uni': "voie partage piéton",
    'cheminement trottoir uni': "voie partage piéton",
    'chemin service site propre uni': "voie partage piéton",
    'chemin dedie uni': "voie partage piéton",
    'autre chemin velo uni': "voie partage piéton",
    'cheminement uni': "voie partage piéton",
    'goulotte': "autre"
}

# Apply the replacement
zone_interest['type voie'] = zone_interest['type voie'].replace(replacement_map)
zone_interest['type voie'] = zone_interest['type voie'].fillna('voie sans aménagement')


In [229]:
point = [48.912196, 2.373194]


centroid = gpd.GeoSeries(
    Point(point[1], point[0]),
    crs="EPSG:4326"
)

centroid = centroid.to_crs("EPSG:2154")
buffer = centroid.buffer(2000)

zone_velo_Saint_Denis = zone_interest.to_crs("EPSG:2154")
zone_velo_Saint_Denis = zone_velo_Saint_Denis[zone_velo_Saint_Denis.intersects(buffer.iloc[0])]

zone_velo_Saint_Denis = zone_velo_Saint_Denis.to_crs(epsg=4326)
buffer = buffer.to_crs(epsg=4326)

In [230]:
import xyzservices.providers as xyz
zone_velo_Saint_Denis.explore(tiles='Stadia.OSMBright',
                       column="type voie",
    categorical=True,
    cmap="tab10",
    legend=True ,
                              location=point,
                              zoom_start=15,


                   figsize=(8, 8))

In [231]:
import plotly.express as px
fig = px.pie(zone_velo_Saint_Denis , values='longueur', names='type voie', title='VOie par type')
fig.show()